![](ubc_header.png)

# Climate-Friendly Food Systems (CFFS) Labelling Project

### The University of British Columbia

****

## Part I: Data Preprocessing

## Set up and Import Libraries

In [1]:
#pip install -r requirements.txt

In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import glob
import os
import csv
from itertools import islice
from decimal import Decimal
import xml.etree.ElementTree as et
from xml.etree.ElementTree import parse
import openpyxl
import pytest

In [3]:
# RUN ONLY ONCE
# os.chdir is used to change the current directory to the specified path
os.chdir("../") # Sets path to the repo folder as it is one level above where this file exists!
path = os.getcwd()
print(path)

/Users/vivaanwadhwa/Documents/GitHub/CFFS_sharon_2024


****

## Load Data Files

### Set Data File Path

In [4]:
# Select data file path for the chosen venue and time range where the recipes data stored

# OK 2023
# filepath_list = glob.glob(os.path.join(os.getcwd(), "data", "raw", "OK 23-24 Sep-Dec*", "*.oc"))

# Totem 2023
# filepath_list += glob.glob(os.path.join(os.getcwd(), "data", "raw", "Feast 23-24 Sep-Dec*", "*.oc"))

# Gather 2023
# filepath_list += glob.glob(os.path.join(os.getcwd(), "data", "raw", "Gather 23-24 Sep-Dec*", "*.oc"))

filepath_list = glob.glob(os.path.join(os.getcwd(), "data", "raw", "BLUEBERRY_FESTIVAL_24"))

filepath_list

['/Users/vivaanwadhwa/Documents/GitHub/CFFS_sharon_2024/data/raw/BLUEBERRY_FESTIVAL_24']

### Import Items List

In [5]:
# Read items.xml files in the filepath_list and construct a dataframe
ItemId = []
Description = []
CaseQty = []
CaseUOM = []
PakQty = []
PakUOM = []
InventoryGroup = []


# from the items xml file, findtext of CaseQty, CaseUOM, PakQty, PakUOM, and InventoryGroup
# then append it on the lists above

for filepath in filepath_list:
    path = filepath + '/Items.xml'
    if os.path.isfile(path):
        xtree = et.parse(path)
        xroot = xtree.getroot()
        for item in xtree.iterfind('Item'):
            ItemId.append(item.attrib['id'])
            Description.append(item.findtext('Description'))
            CaseQty.append(item.findtext('CaseQty'))
            CaseUOM.append(item.findtext('CaseUOM'))
            PakQty.append(item.findtext('PakQty'))
            PakUOM.append(item.findtext('PakUOM'))
            InventoryGroup.append(item.findtext('InventoryGroup'))

            
# Create a dataframe from the lists created above.
        
Items = pd.DataFrame({'ItemId': ItemId, 'Description': Description, 'CaseQty': CaseQty, 
                      'CaseUOM': CaseUOM, 'PakQty': PakQty, 'PakUOM': PakUOM, 'InventoryGroup': InventoryGroup}
                    )
print(Items)
Items.drop_duplicates(inplace=True)

Items.reset_index(drop=True, inplace=True)

     ItemId                     Description CaseQty CaseUOM    PakQty PakUOM  \
0   I-11179                     AGAR POWDER   0.400      Kg     1.000     Kg   
1   I-40957          ALMOND SLICED BLANCHED   1.000     5KG  5000.000      g   
2   I-29389            APPLES DICED IQF FRZ   1.000     bag    10.000     Kg   
3   I-40964  BAKING POWD-DBL ACTIO PINNACLE   1.000      cs  5000.000      g   
4    I-3118               BAKING PWDR MAGIC   5.000      Kg     1.000     Kg   
..      ...                             ...     ...     ...       ...    ...   
65  I-18980          WAFFLES BELGIAN LIEGE   36.000      ea     1.000     ea   
66   I-5983               WATER FOR RECIPES   1.000       L     1.000      L   
67   I-4949              WATERMELON SDLS MX   1.000      lb     1.000     lb   
68   I-6850                   YOGURT GREEK*   1.000     TUB  4000.000      g   
69  I-31129        ZEST LEMON SUGARED REPAK   5.000      lb     1.000     lb   

            InventoryGroup  
0         

In [6]:
# creates a new array with unique ItemIds
all_id_list = Items["ItemId"].unique()

In [7]:
# None of the items are egg yolk liq
Items.loc[Items["Description"] == "Egg Yolk Liq"]

,ItemId,Description,CaseQty,CaseUOM,PakQty,PakUOM,InventoryGroup


In [8]:
# Sumplemental option for egg, vegan option
Items.loc[Items["ItemId"] == "I-68700"]

,ItemId,Description,CaseQty,CaseUOM,PakQty,PakUOM,InventoryGroup


In [9]:
# Gives you the list of breads
breadlist = []


# If the items in the Items list is a "LOAF", "SANDWICH", or "BREAD" then we should add it in the breadlist
# We append it onto the breadlist at position row["ItemId"]
for ind, row in Items.iterrows():
    if ("LOAF" or "SANDWICH" "BREAD") in row["Description"]:
        breadlist.append(row["ItemId"])

breadlist

[]

In [10]:
Items = Items[~Items["InventoryGroup"].isin(["DISPOSABLES", "SUPPLY"])]

In [11]:
# Based on info below there are 486 rows and 7 columns
Items.shape

(66, 7)

In [12]:
Items.dtypes

ItemId            object
Description       object
CaseQty           object
CaseUOM           object
PakQty            object
PakUOM            object
InventoryGroup    object
dtype: object

In [13]:
# Save the dataframe to csv
path = os.path.join(os.getcwd(), "data", "preprocessed", "Items_List.csv")
Items.to_csv(path, index = False, header = True)

### Import Ingredients List

In [14]:
# Read ingredients.xml files in the filepath_list and construct a dataframe
IngredientId = []
Conversion = []
InvFactor = []
Qty = []
Recipe = []
Uom = []

# Using the Ingredients XML file, we extract attributes containing ingredients, conversion, invFactor, qty, recipe, and uom. 
# Then we append it onto the IngredientId, Coversion, InvFactor, Qty, Recipe, and Uom lists
# Then we create a dataframe using the lists created. 

for filepath in filepath_list:
    path = filepath + '/Ingredients.xml'
    if os.path.isfile(path):
        xtree = et.parse(path)
        xroot = xtree.getroot()
        for x in xtree.iterfind('Ingredient'):
            IngredientId.append(x.attrib['ingredient'])
            Conversion.append(x.attrib['conversion'])
            InvFactor.append(x.attrib['invFactor'])
            Qty.append(x.attrib['qty'])
            Recipe.append(x.attrib['recipe'])
            Uom.append(x.attrib['uom'])
    
Ingredients = pd.DataFrame({'IngredientId': IngredientId, 'Qty': Qty,'Uom': Uom, 'Conversion': Conversion, 
                      'InvFactor': InvFactor,'Recipe': Recipe}).drop_duplicates()
Ingredients.drop_duplicates(subset=["IngredientId", "Recipe"], inplace=True)

Ingredients.reset_index(drop=True, inplace=True)

In [15]:
Ingredients.loc[Ingredients["IngredientId"] == "I-29389"]

,IngredientId,Qty,Uom,Conversion,InvFactor,Recipe
146,I-29389,120.000,g,0.00100000,0.0422,P-71672


In [16]:
# This will output the IngredientId on the right side and the number of times the ingredient appears in the Ingredients
# dataframe on the left side. 

# The duplicated() method returns a Series with True and False values that describe which rows in the DataFrame are 
# duplicated and not.

check = Ingredients["IngredientId"].duplicated()

# The line below tells us at which index is the ingredient duplicated. For example I-4598 exists on index 2 of the dataframe and
# then again on index 8. So duplicate is first true on index 8, which is why it is printed below. It also appears in a below 
# index so it is printed again with that index number. 
Ingredients["IngredientId"][check]

22     I-3290
23     I-3658
24     I-6020
27     I-3006
31     I-3290
        ...  
155    I-2979
156    I-3006
157    I-3670
159    I-4531
162    I-5983
Name: IngredientId, Length: 78, dtype: object

In [17]:
# We can see from the printed items above that I-64877 does not appear because it does not get duplicated
Ingredients.loc[Ingredients["IngredientId"] == "I-64877"]

,IngredientId,Qty,Uom,Conversion,InvFactor,Recipe


In [18]:
Ingredients

,IngredientId,Qty,Uom,Conversion,InvFactor,Recipe
0,I-11185,1.000,Kg,35.27399000,0.3333,P-18967
1,I-5983,2.000,L,1.00000000,0.6667,P-18967
2,P-18967,1.000,Kg,1.00000000,1.0000,P-245
3,I-3642,1.000,Kg,1000.00000000,0.2500,P-25993
4,I-6026,3.000,Kg,1000.00000000,0.7500,P-25993
...,...,...,...,...,...,...
159,I-4531,1900.000,g,0.00220462,0.4819,P-71673
160,P-71672,2844.000,g,1.00000000,56.8800,P-71674
161,P-71673,3943.000,g,1.00000000,78.8600,P-71674
162,I-5983,15.000,L,1.00000000,0.5556,P-980


In [19]:
# Below we are checking if we can extract an entire recipe. So we can do .loc with the particular recipe id and then 
# print all the data points (ingredientId, Qty, Uom, Conversion, InvFactor) for that recipe. 
Ingredients.loc[Ingredients["Recipe"] == "R-68698"]

,IngredientId,Qty,Uom,Conversion,InvFactor,Recipe


In [20]:
Items

,ItemId,Description,CaseQty,CaseUOM,PakQty,PakUOM,InventoryGroup
0,I-11179,AGAR POWDER,0.400,Kg,1.000,Kg,FOOD - GROCERY
1,I-40957,ALMOND SLICED BLANCHED,1.000,5KG,5000.000,g,BAKING-RAW INGREDIENTS
2,I-29389,APPLES DICED IQF FRZ,1.000,bag,10.000,Kg,PRODUCE
3,I-40964,BAKING POWD-DBL ACTIO PINNACLE,1.000,cs,5000.000,g,BAKING-RAW INGREDIENTS
4,I-3118,BAKING PWDR MAGIC,5.000,Kg,1.000,Kg,FOOD - GROCERY
...,...,...,...,...,...,...,...
65,I-18980,WAFFLES BELGIAN LIEGE,36.000,ea,1.000,ea,FOOD - GROCERY
66,I-5983,WATER FOR RECIPES,1.000,L,1.000,L,BEVERAGE
67,I-4949,WATERMELON SDLS MX,1.000,lb,1.000,lb,PRODUCE
68,I-6850,YOGURT GREEK*,1.000,TUB,4000.000,g,DAIRY


In [21]:
Ingredients.shape

(164, 6)

In [22]:
Ingredients.dtypes

IngredientId    object
Qty             object
Uom             object
Conversion      object
InvFactor       object
Recipe          object
dtype: object

In [23]:
# Save the dataframe to csv
path = os.path.join(os.getcwd(), "data", "preprocessed", "Ingredients_List.csv")
Ingredients.to_csv(path, index = False, header = True)

### Import Preps List

In [24]:
# Read preps.xml files in the filepath_list and construct a dataframe
PrepId = []
Description = []
PakQty = []
PakUOM = []
InventoryGroup = []


# Here we do the same thing for the Preps XML file where we find the columns using attrib function and then append it onto
# the dataframe called Preps. 

for filepath in filepath_list:
    path = filepath + '/Preps.xml'
    if os.path.isfile(path):
        xtree = et.parse(path)
        xroot = xtree.getroot()
        for x in xtree.iterfind('Prep'):
            PrepId.append(x.attrib['id'])
            Description.append(x.findtext('Description'))
            PakQty.append(x.findtext('PakQty'))
            PakUOM.append(x.findtext('PakUOM'))
            InventoryGroup.append(x.findtext('InventoryGroup'))
    
Preps = pd.DataFrame({'PrepId': PrepId, 'Description': Description,
                  'PakQty': PakQty, 'PakUOM':PakUOM, 'InventoryGroup': InventoryGroup}).drop_duplicates()
preps_columns = Preps.columns
Preps.drop_duplicates(subset=["PrepId"], inplace=True)

Preps.reset_index(drop=True, inplace=True)

In [25]:
Preps

,PrepId,Description,PakQty,PakUOM,InventoryGroup
0,P-51178,BF||Blueberrie Chia Scone,25.000,ea,
1,P-71674,BF|BB Oatmeal Bar,50.000,each,
2,P-51184,BF|Blueberry Compote,1.000,L,ZDONT USE FOOD
3,P-51183,BF|Blueberry Yogurt Cup,1.000,ea,
4,P-68421,BF|Cake Blueberry Upsidedown,160.000,PTN,BAKED GOODS
5,P-36196,BF|Cheesecake Blueberry Mango,250.000,ea,
6,P-44402,BF|Cookies Blueberry Power,18.000,ea,
7,P-68423,BF|Crumble Blueberry,1.000,ea,
8,P-68033,BF|Mochi Donut,1.000,each,
9,P-44782,BF|MUFFIN|Blueberry Lemon Oat,65.000,ea,


In [26]:
# This tells us that there are not any duplicates in the Preps dataframe. There is a different process|ingredient pair for 
# all the recipes. 
check = Preps["PrepId"].duplicated().any()
print(check)

False


In [27]:
# There are 546 rows and 5 columns
Preps.shape

(30, 5)

In [28]:
# Here we see there is only one place P-50739 is used in the Preps
Preps.loc[Preps["PrepId"] == "P-50739"]

,PrepId,Description,PakQty,PakUOM,InventoryGroup


In [29]:
Preps.dtypes

PrepId            object
Description       object
PakQty            object
PakUOM            object
InventoryGroup    object
dtype: object

In [30]:
# Save the dataframe to csv
path = os.path.join(os.getcwd(), "data", "preprocessed", "Preps_List.csv")
Preps.to_csv(path, index = False, header = True)

### Import Products List

In [31]:
# Read products.xml files in the filepath_list and construct a dataframe
ProdId = []
Description = []
SalesGroup = []

# From the XML file for products append the id into ProdId, description into Description, and SalesGroup into SalesGroup list.
# Make a dataframe out of the 3 lists
# Then also drop the duplicates in the Products dataframe

for filepath in filepath_list:
    path = filepath + '/Products.xml'
    if os.path.isfile(path):
        xtree = et.parse(path)
        xroot = xtree.getroot()
        for x in xtree.iterfind('Prod'):
            ProdId.append(x.attrib['id'])
            Description.append(x.findtext('Description'))
            SalesGroup.append(x.findtext('SalesGroup'))
        
Products = pd.DataFrame({'ProdId': ProdId, 'Description': Description, 'SalesGroup': SalesGroup})
Products.drop_duplicates(inplace=True)

Products.reset_index(drop=True, inplace=True)

In [32]:
Products

,ProdId,Description,SalesGroup


In [33]:
# Here we can see that there is only one R-56966 in the products 
Products.loc[Products["ProdId"] == "R-56966"]

,ProdId,Description,SalesGroup


In [34]:
Products.dtypes

ProdId         float64
Description    float64
SalesGroup     float64
dtype: object

In [35]:
# Here we can see that there is only one R-68698 in the products 
Products.loc[Products["ProdId"] == "R-68698"]

,ProdId,Description,SalesGroup


In [36]:
Products.shape

(0, 3)

In [37]:
Products.dtypes

ProdId         float64
Description    float64
SalesGroup     float64
dtype: object

In [38]:
# Save the dataframe to csv
path = os.path.join(os.getcwd(), "data", "preprocessed", "Products_List.csv")
Products.to_csv(path, index = False, header = True)

### Import Conversions List

In [39]:
# Read conventions.xml files in the filepath_list and construct a dataframe
ConversionId = []
Multiplier = []
ConvertFromQty = []
ConvertFromUom = []
ConvertToQty = []
ConvertToUom = []

# From the XML file for Conversions append the id into ConversionId, multiplier into Multiplier, ConvertFrom->qty into 
# ConvertFromQty,ConvertFrom->uom into ConvertFromUom, ConvertTo->qty into ConvertToQty and and ConvertTo->uom into the
# CovertToUom list. 
# Make a dataframe out of the 3 lists
# Then also drop the duplicates in the Products dataframe


for filepath in filepath_list:
    path = filepath + '/Conversions.xml'
    if os.path.isfile(path):
        xtree = et.parse(path)
        xroot = xtree.getroot()
        for x in xtree.iterfind('Conversion'):
            ConversionId.append(x.attrib['id'])
            Multiplier.append(x.attrib['multiplier'])
            ConvertFromQty.append(x.find('ConvertFrom').attrib['qty'])
            ConvertFromUom.append(x.find('ConvertFrom').attrib['uom'])
            ConvertToQty.append(x.find('ConvertTo').attrib['qty'])
            ConvertToUom.append(x.find('ConvertTo').attrib['uom'])
    
    
Conversions = pd.DataFrame({'ConversionId': ConversionId, 'Multiplier': Multiplier, 'ConvertFromQty': ConvertFromQty,
                           'ConvertFromUom': ConvertFromUom, 'ConvertToQty': ConvertToQty, 'ConvertToUom': ConvertToUom}
                          ).drop_duplicates()

Conversions.reset_index(drop=True, inplace=True)

In [40]:
# Here we can see for example that to convert 1.14 L to 1 L the multiplier is 0.877 since 1/1.14 = 0.877
Conversions

,ConversionId,Multiplier,ConvertFromQty,ConvertFromUom,ConvertToQty,ConvertToUom
0,,1.00000000,1.0000,XXX,1.0000,L
1,,0.87719298,1.0000,1.14L,1.1400,L
2,,0.66666667,1.0000,1.5L,1.5000,L
3,,0.57142857,1.0000,1.75 L,1.7500,L
4,,0.50000000,1.0000,2L,2.0000,L
...,...,...,...,...,...,...
63,I-5983,0.03378378,1.0000,fl oz,29.6000,g
64,I-5983,0.00100000,1.0000,L,1000.0000,g
65,I-6791,0.00411523,1.0000,cup,243.0000,g
66,I-8058,0.00408163,1.0000,cup,245.0000,g


In [41]:
# Here we can check that there are no ingredients listed since 
Conversions.loc[Conversions["ConversionId"] == "I-4582"]

,ConversionId,Multiplier,ConvertFromQty,ConvertFromUom,ConvertToQty,ConvertToUom


In [42]:
all_id_list = Items["ItemId"].unique()
all_conv_list = Conversions["ConversionId"].unique()

print("All unique IDs list\n")
print(all_id_list)
print("\n")
print("All unique Conversions list\n")
print(all_conv_list)

All unique IDs list

['I-11179' 'I-40957' 'I-29389' 'I-40964' 'I-3118' 'I-3119' 'I-4507'
 'I-4530' 'I-33601' 'I-4531' 'I-4534' 'I-6020' 'I-11433' 'I-19143'
 'I-3222' 'I-8325' 'I-2979' 'I-40951' 'I-3225' 'I-1472' 'I-2157' 'I-2156'
 'I-29032' 'I-3283' 'I-2986' 'I-3290' 'I-3292' 'I-11189' 'I-23753'
 'I-4617' 'I-4772' 'I-1813' 'I-52598' 'I-45822' 'I-8058' 'I-6791' 'I-786'
 'I-3364' 'I-24931' 'I-3373' 'I-3006' 'I-56382' 'I-3642' 'I-3588'
 'I-11185' 'I-42143' 'I-3497' 'I-6026' 'I-12176' 'I-3501' 'I-2254'
 'I-6820' 'I-3658' 'I-3660' 'I-6027' 'I-35921111' 'I-12' 'I-3669' 'I-3670'
 'I-2784' 'I-3498' 'I-18980' 'I-5983' 'I-4949' 'I-6850' 'I-31129']


All unique Conversions list

['' 'I-1813' 'I-2254' 'I-2384' 'I-3225' 'I-3283' 'I-3497' 'I-3501'
 'I-3642' 'I-3658' 'I-3660' 'I-3669' 'I-3670' 'I-4507' 'I-4530' 'I-4772'
 'I-4949' 'I-5983' 'I-6791' 'I-8058']


In [43]:
# Here we have the number of items in the list that are a part of the "all_id_list" but not part of the "all_conv_list"
missing_conv_id = []

for item in all_id_list:
    if item not in all_conv_list:
        missing_conv_id.append(item)
        
missing_conv_id
print(len(missing_conv_id))

48


In [44]:
Conversions.shape

(68, 6)

In [45]:
Conversions.loc[Conversions["ConversionId"] == "I-29389"]

,ConversionId,Multiplier,ConvertFromQty,ConvertFromUom,ConvertToQty,ConvertToUom


In [46]:
Conversions.dtypes

ConversionId      object
Multiplier        object
ConvertFromQty    object
ConvertFromUom    object
ConvertToQty      object
ConvertToUom      object
dtype: object

In [47]:
# Save the dataframe to csv
path = os.path.join(os.getcwd(), "data", "preprocessed", "Conversions_List.csv")
Conversions.to_csv(path, index = False, header = True)

***
## Data Summary

In [48]:
# Summary of raw data imported for evaluation
# Here we have a summary of the number of items, preps, ingredients, products, conversions

datasum = pd.DataFrame([Items.shape, Preps.shape, Ingredients.shape, Products.shape, Conversions.shape],
                       columns = ['count', 'columns'], 
                       index = ['Items', 'Preps', 'Ingredients', 'Products', 'Conversions'])
datasum

,count,columns
Items,66,7
Preps,30,5
Ingredients,164,6
Products,0,3
Conversions,68,6
